# 本ノートブックの目的
自然言語処理により文章を分析するための技術検証を行う。

## 感情分析
文章のポジティブ・ネガティブを分析する。

In [1]:
from transformers import pipeline
import fugashi
from pprint import pprint
import time
import pandas as pd

In [2]:
# モデル構築
text_classification_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-marc_ja"
)

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.0.1)
    Python  3.11.4 (you have 3.11.4)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [3]:
target_data = "data/negapozi.xlsx"
df = pd.read_excel(target_data)
df

,id,文章
0,1,ケータイを充電する際は、過充電に注意が必要です。バッテリーは、過充電によって劣化が早まる可能...
1,2,自分を育ててくれた親に対して恩返しの気持ちで介護しているかもしれませんが、介護は体力的にも精...
2,3,どれだけ顔や体のラインが綺麗でも、姿勢が悪いとそれだけで美しさが損なわれてしまいます。
3,4,バッテリーに対して過充電の負荷をかけないことで、ケータイを長く大切に使用できます。
4,5,ご本人様・ご家族の皆様が仲良く笑顔でいられますよう、専門スタッフがサポートさせていただきます。


In [6]:
text_list = []
for i in df.index:
    text_list.append(text_classification_pipeline(df["文章"][i])[0])
judge_df = pd.DataFrame(text_list)
judge_df = judge_df.rename(columns={"label": "p/n"})
judge_df

,p/n,score
0,negative,0.837943
1,positive,0.787296
2,negative,0.746773
3,positive,0.999411
4,positive,0.998665


In [7]:
# データフレーム結合
negapozi = pd.merge(df, judge_df, how="left", right_index=True, left_index=True)
negapozi["id"] = negapozi["id"].map(lambda x: str(x).zfill(4))
negapozi

,id,文章,p/n,score
0,0001,ケータイを充電する際は、過充電に注意が必要です。バッテリーは、過充電によって劣化が早まる可能...,negative,0.837943
1,0002,自分を育ててくれた親に対して恩返しの気持ちで介護しているかもしれませんが、介護は体力的にも精...,positive,0.787296
2,0003,どれだけ顔や体のラインが綺麗でも、姿勢が悪いとそれだけで美しさが損なわれてしまいます。,negative,0.746773
3,0004,バッテリーに対して過充電の負荷をかけないことで、ケータイを長く大切に使用できます。,positive,0.999411
4,0005,ご本人様・ご家族の皆様が仲良く笑顔でいられますよう、専門スタッフがサポートさせていただきます。,positive,0.998665


In [8]:
negapozi.to_excel('result/negapozi_judge.xlsx', index=False)

In [ ]:
positive_text = "世界には言葉がわからなくても感動する音楽がある。"
# positive_textの極性を予測
print(text_classification_pipeline(positive_text)[0])

In [ ]:
negative_text = "私は今日は仕事にやる気が出ないです。"
# positive_textの極性を予測
before = time.time()
print(text_classification_pipeline(negative_text)[0])
after = time.time()
print("it took {}sec".format(after - before))

## 要約生成

In [ ]:
# モデル構築
text2text_pipeline = pipeline(
    model="llm-book/t5-base-long-livedoor-news-corpus"
)

In [ ]:
# articleの要約を生成
article = "ついに始まった３連休。テレビを見ながら過ごしている人も多いのではないだろうか？　今夜オススメなのは何と言っても、NHKスペシャル「世界を変えた男 スティーブ・ジョブズ」だ。実は知らない人も多いジョブズ氏の養子に出された生い立ちや、アップル社から一時追放されるなどの経験。そして、彼が追い求めた理想の未来とはなんだったのか、ファンならずとも気になる内容になっている。 今年、亡くなったジョブズ氏の伝記は日本でもベストセラーになっている。今後もアップル製品だけでなく、世界でのジョブズ氏の影響は大きいだろうと想像される。ジョブズ氏のことをあまり知らないという人もこの機会にぜひチェックしてみよう。 世界を変えた男　スティーブ・ジョブズ（NHKスペシャル）"
before = time.time()
print(text2text_pipeline(article)[0]["generated_text"])
after = time.time()
print("it took {}sec".format(after - before))

## 固有表現認識

In [ ]:
# モデル構築
ner_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-ner-wikipedia-dataset",
    aggregation_strategy="simple",
)

In [ ]:
text = "大谷翔平は岩手県水沢市出身のプロ野球選手"
# text中の固有表現を抽出
before = time.time()
pprint(ner_pipeline(text))
after = time.time()
print("it took {}sec".format(after - before))

In [ ]:
意味類似度解析

In [ ]:
text_sim_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-jsts",
    function_to_apply="none",
)
text = "川べりでサーフボードを持った人たちがいます"
sim_text = "サーファーたちが川べりに立っています"
# textとsim_textの類似度を計算
result = text_sim_pipeline({"text": text, "text_pair": sim_text})
print(result["score"])

In [ ]:
dissim_text = "トイレの壁に黒いタオルがかけられています"
# textとdissim_textの類似度を計算
result = text_sim_pipeline({"text": text, "text_pair": dissim_text})
print(result["score"])

In [ ]:
from torch.nn.functional import cosine_similarity

sim_enc_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-unsup-simcse-jawiki",
    task="feature-extraction",
)

# textとsim_textのベクトルを獲得
text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb = sim_enc_pipeline(sim_text, return_tensors=True)[0][0]
# textとsim_textの類似度を計算
sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())

In [ ]:
# dissim_textのベクトルを獲得
dissim_emb = sim_enc_pipeline(dissim_text, return_tensors=True)[0][0]
# textとdissim_textの類似度を計算
dissim_pair_score = cosine_similarity(text_emb, dissim_emb, dim=0)
print(dissim_pair_score.item())